# Register ADK Agent On Vertex AI Agent Engine With Gemini Enterprise

This notebook registers a deployed ADK agent with Gemini Enterprise (formerly known as Agentspace), making it available in the Gemini Enterprise interface.

**Prerequisites:**
- Agent must be deployed using [deploy-vertex-ai-agent-engine.ipynb](./deploy-vertex-ai-agent-engine.ipynb)
- `deployment.json` file exists in this folder with the deployment resource ID
- Gemini Enterprise must be enabled in your Google Cloud project

## Package Imports

In [1]:
import os
import subprocess
import json
from pathlib import Path
from datetime import datetime

import dotenv
import requests
from vertexai import agent_engines

## Auto-Configure Paths

In [2]:
# Get current directory (should be agent_name/deploy/)
deploy_folder = Path.cwd()
agent_folder = deploy_folder.parent
project_root = agent_folder.parent

# Extract agent name from folder structure
agent_name = agent_folder.name

print(f"Deploy folder: {deploy_folder}")
print(f"Agent folder: {agent_folder}")
print(f"Project root: {project_root}")
print(f"Agent name: {agent_name}")

Deploy folder: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Applied ML/AI Agents/concept-bq/agent_bq_forecast/deploy
Agent folder: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Applied ML/AI Agents/concept-bq/agent_bq_forecast
Project root: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Applied ML/AI Agents/concept-bq
Agent name: agent_bq_forecast


## Get Google Cloud Project Information

In [3]:
# Get project ID and number from gcloud
PROJECT_ID = subprocess.run(
    ['gcloud', 'config', 'get-value', 'project'],
    capture_output=True,
    text=True,
    check=True
).stdout.strip()

PROJECT_NUMBER = subprocess.run(
    ['gcloud', 'projects', 'describe', PROJECT_ID, '--format=value(projectNumber)'],
    capture_output=True,
    text=True,
    check=True
).stdout.strip()

print(f"\n{'='*50}")
print("Google Cloud Project Information")
print(f"{'='*50}")
print(f"PROJECT_ID     = {PROJECT_ID}")
print(f"PROJECT_NUMBER = {PROJECT_NUMBER}")
print(f"{'='*50}\n")


Google Cloud Project Information
PROJECT_ID     = statmike-mlops-349915
PROJECT_NUMBER = 1026793852137



## Load Deployment Metadata

Get the deployed agent's information from `deployment.json`.

In [4]:
deployment_file = deploy_folder / 'deployment.json'

if not deployment_file.exists():
    raise FileNotFoundError(
        f"Deployment metadata not found at {deployment_file}. "
        "Please deploy the agent first using deploy-vertex-ai-agent-engine.ipynb"
    )

with open(deployment_file, 'r') as f:
    deployment_metadata = json.load(f)

current_deployment = deployment_metadata.get('resource_id')

if not current_deployment:
    raise ValueError("No deployment resource_id found in deployment.json")

print(f"Deployment Resource ID: {current_deployment}")

Deployment Resource ID: projects/1026793852137/locations/us-central1/reasoningEngines/2255516162987130880


## Get ADK Agent on Vertex AI Agent Engine

In [5]:
try:
    remote_app = agent_engines.get(resource_name=current_deployment)
    print(f"✓ Connected to deployed agent: {remote_app.resource_name}")
except Exception as e:
    raise RuntimeError(f"Failed to connect to deployed agent: {e}")

✓ Connected to deployed agent: projects/1026793852137/locations/us-central1/reasoningEngines/2255516162987130880


In [6]:
remote_app.resource_name

'projects/1026793852137/locations/us-central1/reasoningEngines/2255516162987130880'

In [7]:
remote_app.display_name

'agent_bq_forecast'

In [8]:
remote_app.to_dict()['description']

'An agent that can use bigquery with tools to answer questions with time series data including forecasting.'

## Configure Gemini Enterprise Settings

**IMPORTANT:** You need to update the `APP_ID` below with your Gemini Enterprise app ID.

To find your app ID:
1. Go to [Gemini Enterprise Console](https://console.cloud.google.com/gen-app-builder/)
2. Select your app
3. The app ID is in the URL or app details

The location is typically `global` for Gemini Enterprise.

In [9]:
LOCATION = "global"  # Or your specific location
AGENT_DISPLAY_NAME = remote_app.display_name
AGENT_DESCRIPTION = remote_app.to_dict()['description']

# UPDATE THIS with your Gemini Enterprise app ID
APP_ID = "enterprise-search-17381091_1738109128316"  # e.g., "enterprise-search-17381091_1738109128316"

if APP_ID == "YOUR_GEMINI_ENTERPRISE_APP_ID":
    raise ValueError(
        "Please update APP_ID with your actual Gemini Enterprise app ID. "
        "See the markdown cell above for instructions."
    )

print(f"Location: {LOCATION}")
print(f"Agent Display Name: {AGENT_DISPLAY_NAME}")
print(f"Agent Description: {AGENT_DESCRIPTION}")
print(f"App ID: {APP_ID}")

Location: global
Agent Display Name: agent_bq_forecast
Agent Description: An agent that can use bigquery with tools to answer questions with time series data including forecasting.
App ID: enterprise-search-17381091_1738109128316


## Check for Existing Registration

List all agents in Gemini Enterprise and check if one with the same display name already exists.

In [10]:
# Get access token
access_token = subprocess.check_output(
    ['gcloud', 'auth', 'print-access-token'],
    text=True
).strip()

# List all agents in this Gemini Enterprise app
list_url = (
    f"https://discoveryengine.googleapis.com/v1alpha/"
    f"projects/{PROJECT_NUMBER}/locations/{LOCATION}/"
    f"collections/default_collection/engines/{APP_ID}/"
    f"assistants/default_assistant/agents"
)

print(f"Listing agents at: {list_url}")

list_response = requests.get(
    url=list_url,
    headers={
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "X-Goog-User-Project": PROJECT_NUMBER
    },
)

print(f"Status Code: {list_response.status_code}")

if list_response.status_code != 200:
    print(f"Failed to list agents: {list_response.text}")
    raise RuntimeError(f"Failed to list agents: {list_response.text}")

# Check if our agent already exists
agents = list_response.json().get('agents', [])
print(f"\nFound {len(agents)} agent(s) in Gemini Enterprise")

existing_agent = None
for agent in agents:
    if agent.get('displayName') == AGENT_DISPLAY_NAME:
        existing_agent = agent
        print(f"\n✓ Found existing agent: {agent['name']}")
        break

if not existing_agent:
    print(f"\nNo existing agent found with displayName: {AGENT_DISPLAY_NAME}")

Listing agents at: https://discoveryengine.googleapis.com/v1alpha/projects/1026793852137/locations/global/collections/default_collection/engines/enterprise-search-17381091_1738109128316/assistants/default_assistant/agents
Status Code: 200

Found 5 agent(s) in Gemini Enterprise

No existing agent found with displayName: agent_bq_forecast


## Register Agent with Gemini Enterprise

This makes a REST API call to register the deployed ADK agent with Gemini Enterprise (only if not already registered).

In [11]:
if existing_agent:
    print("Agent is already registered. Using existing registration.")
    agent_data = existing_agent
else:
    print("Creating new agent registration...")
    
    # Construct API endpoint
    create_url = (
        f"https://discoveryengine.googleapis.com/v1alpha/"
        f"projects/{PROJECT_NUMBER}/locations/{LOCATION}/"
        f"collections/default_collection/engines/{APP_ID}/"
        f"assistants/default_assistant/agents"
    )
    
    # Prepare request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "X-Goog-User-Project": PROJECT_NUMBER
    }
    
    payload = {
        "displayName": AGENT_DISPLAY_NAME,
        "description": AGENT_DESCRIPTION,
        "adk_agent_definition": {
            "tool_settings": {
                "tool_description": AGENT_DESCRIPTION
            },
            "provisioned_reasoning_engine": {
                "reasoning_engine": remote_app.resource_name
            }
        },
    }
    
    # Make request
    create_response = requests.post(create_url, headers=headers, json=payload)
    
    if create_response.status_code != 200:
        print(f"\n✗ Registration failed: {create_response.status_code}")
        print(create_response.text)
        raise RuntimeError(f"Failed to register agent: {create_response.text}")
    
    agent_data = create_response.json()
    print("\n✓ Agent successfully registered!")

# Display agent details
print("\nAgent Details:")
print(json.dumps(agent_data, indent=2))

Creating new agent registration...

✓ Agent successfully registered!

Agent Details:
{
  "name": "projects/1026793852137/locations/global/collections/default_collection/engines/enterprise-search-17381091_1738109128316/assistants/default_assistant/agents/9862578113183617741",
  "displayName": "agent_bq_forecast",
  "description": "An agent that can use bigquery with tools to answer questions with time series data including forecasting.",
  "createTime": "2025-10-19T20:09:17.486065859Z",
  "adkAgentDefinition": {
    "toolSettings": {
      "toolDescription": "An agent that can use bigquery with tools to answer questions with time series data including forecasting."
    },
    "provisionedReasoningEngine": {
      "reasoningEngine": "projects/1026793852137/locations/us-central1/reasoningEngines/2255516162987130880"
    }
  },
  "state": "ENABLED"
}


## Save Gemini Enterprise Registration Info

Update `deployment.json` with the Gemini Enterprise agent ID.

In [12]:
# Extract agent info from agent data
gemini_agent_name = agent_data.get('name', '')
create_time = agent_data.get('createTime', '')

# Update deployment metadata
deployment_metadata['gemini_enterprise_agent_id'] = gemini_agent_name
deployment_metadata['registered_at'] = create_time

# Save updated metadata
with open(deployment_file, 'w') as f:
    json.dump(deployment_metadata, f, indent=2)

print(f"\n✓ Registration info saved to {deployment_file}")
print(f"\nGemini Enterprise Agent:")
print(f"  Name: {gemini_agent_name}")
print(f"  Display Name: {agent_data.get('displayName')}")
print(f"  Created: {create_time}")
print(f"  State: {agent_data.get('state')}")


✓ Registration info saved to /usr/local/google/home/statmike/Git/vertex-ai-mlops/Applied ML/AI Agents/concept-bq/agent_bq_forecast/deploy/deployment.json

Gemini Enterprise Agent:
  Name: projects/1026793852137/locations/global/collections/default_collection/engines/enterprise-search-17381091_1738109128316/assistants/default_assistant/agents/9862578113183617741
  Display Name: agent_bq_forecast
  Created: 2025-10-19T20:09:17.486065859Z
  State: ENABLED


---
## Delete Agent from Gemini Enterprise

Use this section to remove the agent registration from Gemini Enterprise.

**⚠️ Warning:** This only removes the registration from Gemini Enterprise. It does NOT delete the deployment on Vertex AI Agent Engine.

In [13]:
# Set to True to delete the agent registration from Gemini Enterprise
delete_agent = False

if delete_agent:
    if not gemini_agent_name:
        print("✗ No agent to delete - gemini_agent_name not found")
    else:
        print(f"Deleting agent: {gemini_agent_name}")
        
        # Refresh access token
        access_token = subprocess.check_output(
            ['gcloud', 'auth', 'print-access-token'],
            text=True
        ).strip()
        
        # Delete the agent
        delete_response = requests.delete(
            url=f"https://discoveryengine.googleapis.com/v1alpha/{gemini_agent_name}",
            headers={
                "Authorization": f"Bearer {access_token}",
                "Content-Type": "application/json",
                "X-Goog-User-Project": PROJECT_NUMBER
            },
            json={
                "name": gemini_agent_name
            }
        )
        
        print(f"Status Code: {delete_response.status_code}")
        
        if delete_response.status_code == 200:
            print("✓ Agent successfully deleted from Gemini Enterprise")
            
            # Clear metadata
            deployment_metadata['gemini_enterprise_agent_id'] = ''
            deployment_metadata['registered_at'] = ''
            
            with open(deployment_file, 'w') as f:
                json.dump(deployment_metadata, f, indent=2)
            
            print(f"✓ Registration metadata cleared from {deployment_file}")
        else:
            print(f"✗ Deletion failed: {delete_response.status_code}")
            if delete_response.text:
                print(f"Response: {delete_response.text}")
            else:
                print("Response body is empty")
else:
    print("Deletion skipped (delete_agent = False)")

Deletion skipped (delete_agent = False)
